## 0. Installing Huggin Face's Transformers and Loading Dependencies

In [1]:
!pip install transformers

In [2]:
from transformers import pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


## 1. Data Preprocessing: Extracting transcript from a youtube video url

In [6]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00


In [7]:
from youtube_transcript_api import YouTubeTranscriptApi

In [34]:
video = "https://www.youtube.com/watch?v=JctmnczWg0U"

In [35]:
video_id = video.split("=")[1]
video_id

'JctmnczWg0U'

In [36]:
transcript = YouTubeTranscriptApi.get_transcript(video_id)
transcript

[{'text': "nick i'm going to need a summary of",
  'start': 0.0,
  'duration': 3.28},
 {'text': 'these papers for the japan report by',
  'start': 1.439,
  'duration': 4.721},
 {'text': 'close of business today', 'start': 3.28, 'duration': 2.88},
 {'text': "uh there's like 500 pages here", 'start': 8.559, 'duration': 6.96},
 {'text': 'perfect thanks knew i could count on you',
  'start': 12.719,
  'duration': 4.081},
 {'text': "we're gonna need some help", 'start': 15.519, 'duration': 3.121},
 {'text': 'luckily we can use hugging face', 'start': 16.8, 'duration': 4.479},
 {'text': 'transformers for ai based summarization',
  'start': 18.64,
  'duration': 4.799},
 {'text': "let's do it tired of reading a ton of",
  'start': 21.279,
  'duration': 4.0},
 {'text': "blog posts well in this video we're",
  'start': 23.439,
  'duration': 3.361},
 {'text': 'going to be taking a look at how we can',
  'start': 25.279,
  'duration': 1.84},
 {'text': 'use', 'start': 26.8, 'duration': 3.76},
 {'te

In [37]:
doc = []
for line in transcript:
  if "\n" in line['text']:
    x = line["text"].replace("\n", " ")
    doc.append(x)
  else:
    doc.append(line['text'])

print(doc)

["nick i'm going to need a summary of", 'these papers for the japan report by', 'close of business today', "uh there's like 500 pages here", 'perfect thanks knew i could count on you', "we're gonna need some help", 'luckily we can use hugging face', 'transformers for ai based summarization', "let's do it tired of reading a ton of", "blog posts well in this video we're", 'going to be taking a look at how we can', 'use', 'ai to summarize long blog posts', "so for this we're going to be using", 'hugging face and a number of natural', 'language processing techniques', "let's take a deeper look as to what", "we're going to be going through so in", "order to summarize our blog post we're", 'going to be using a library called', 'transformers', 'by a group called hugging face and', "specifically we're going to be using", 'their summarization pipelines capability', 'so this allows you to', 'pass your blocker text and have it', 'summarize now because there is a bit of', 'a limit on that pipeline

In [38]:
paragraph = " ".join(doc)
paragraph

"nick i'm going to need a summary of these papers for the japan report by close of business today uh there's like 500 pages here perfect thanks knew i could count on you we're gonna need some help luckily we can use hugging face transformers for ai based summarization let's do it tired of reading a ton of blog posts well in this video we're going to be taking a look at how we can use ai to summarize long blog posts so for this we're going to be using hugging face and a number of natural language processing techniques let's take a deeper look as to what we're going to be going through so in order to summarize our blog post we're going to be using a library called transformers by a group called hugging face and specifically we're going to be using their summarization pipelines capability so this allows you to pass your blocker text and have it summarize now because there is a bit of a limit on that pipeline what we need to do is a little bit of processing in order to be able to handle la

In [39]:
paragraph = paragraph.replace(".", ".<eos>")
paragraph = paragraph.replace("!", "!<eos>")
paragraph = paragraph.replace("?", "?<eos>")
sentences = paragraph.split("<eos>")
sentences

["nick i'm going to need a summary of these papers for the japan report by close of business today uh there's like 500 pages here perfect thanks knew i could count on you we're gonna need some help luckily we can use hugging face transformers for ai based summarization let's do it tired of reading a ton of blog posts well in this video we're going to be taking a look at how we can use ai to summarize long blog posts so for this we're going to be using hugging face and a number of natural language processing techniques let's take a deeper look as to what we're going to be going through so in order to summarize our blog post we're going to be using a library called transformers by a group called hugging face and specifically we're going to be using their summarization pipelines capability so this allows you to pass your blocker text and have it summarize now because there is a bit of a limit on that pipeline what we need to do is a little bit of processing in order to be able to handle l

In [40]:
len(sentences)

10

In [41]:
sentences[1]

"com forward slash long form summarize a long form dash summarization dash with hugging face and you should be able to get this entire notebook available pretty easily but as always we're going to go through this step by step and actually take a look at how it's done now also another thing to note whilst we're doing this on blog posts you could do this on a whole bunch of other different types of text if you wanted to so if you wanted to summarize research papers or newspaper articles you could definitely do that as well alrighty on to step one so first up what we're going to do is install our core dependency which is going to be transformers so let's go ahead and do it alrighty so that's our installation done so in order to do that what we've written is exclamation mark pip install transformers so this is going to go on ahead and install the transformers library into our python environment so we've now got that available now the next thing that we need to do is actually go ahead and i

### Chunking our data

In [54]:
max_chunk = 500
curr_chunk = 0
chunks = []
for sentence in sentences:
    words = sentence.split(" ")
    if chunks and len(chunks[curr_chunk]) + len(words) <= max_chunk:
        chunks[curr_chunk].extend(words)
    else:
        i = 0
        while i < len(words):
            chunk_size = min(max_chunk, len(words) - i)
            chunks.append(words[i:i + chunk_size])
            i += chunk_size
            curr_chunk = len(chunks) - 1


In [55]:
for chunk in chunks:
  print(len(chunk))

500
329
500
500
500
500
116
500
249
500
459
500
192
488
44
500
101


In [56]:
chunks[0]

['nick',
 "i'm",
 'going',
 'to',
 'need',
 'a',
 'summary',
 'of',
 'these',
 'papers',
 'for',
 'the',
 'japan',
 'report',
 'by',
 'close',
 'of',
 'business',
 'today',
 'uh',
 "there's",
 'like',
 '500',
 'pages',
 'here',
 'perfect',
 'thanks',
 'knew',
 'i',
 'could',
 'count',
 'on',
 'you',
 "we're",
 'gonna',
 'need',
 'some',
 'help',
 'luckily',
 'we',
 'can',
 'use',
 'hugging',
 'face',
 'transformers',
 'for',
 'ai',
 'based',
 'summarization',
 "let's",
 'do',
 'it',
 'tired',
 'of',
 'reading',
 'a',
 'ton',
 'of',
 'blog',
 'posts',
 'well',
 'in',
 'this',
 'video',
 "we're",
 'going',
 'to',
 'be',
 'taking',
 'a',
 'look',
 'at',
 'how',
 'we',
 'can',
 'use',
 'ai',
 'to',
 'summarize',
 'long',
 'blog',
 'posts',
 'so',
 'for',
 'this',
 "we're",
 'going',
 'to',
 'be',
 'using',
 'hugging',
 'face',
 'and',
 'a',
 'number',
 'of',
 'natural',
 'language',
 'processing',
 'techniques',
 "let's",
 'take',
 'a',
 'deeper',
 'look',
 'as',
 'to',
 'what',
 "we're",


In [57]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id] = " ".join(chunks[chunk_id])
chunks[0]

"nick i'm going to need a summary of these papers for the japan report by close of business today uh there's like 500 pages here perfect thanks knew i could count on you we're gonna need some help luckily we can use hugging face transformers for ai based summarization let's do it tired of reading a ton of blog posts well in this video we're going to be taking a look at how we can use ai to summarize long blog posts so for this we're going to be using hugging face and a number of natural language processing techniques let's take a deeper look as to what we're going to be going through so in order to summarize our blog post we're going to be using a library called transformers by a group called hugging face and specifically we're going to be using their summarization pipelines capability so this allows you to pass your blocker text and have it summarize now because there is a bit of a limit on that pipeline what we need to do is a little bit of processing in order to be able to handle la

In [58]:
len(chunks)

17

In [72]:
res = summarizer(chunks, max_length=50, min_length=15, do_sample=False)

In [73]:
len(res)

17

In [74]:
res

[{'summary_text': " In this video we're going to be taking a look at how we can use ai to summarize long blog posts . In order to summarize our blog post we're using a library called transformers by a group called hugging face . We"},
 {'summary_text': " The code for this video is going to be available via GitHub so if you want to pick this up and just run with it really quickly you can go to GitHub . We're going to use beautiful soup to pre-process a blog post"},
 {'summary_text': " We're going to go through this step by step and actually take a look at how it's done . We'll install our core dependency which is going to be transformers so in order to do that what we've written is exclamation"},
 {'summary_text': " we actually wrote so what we've done is we've created a new variable called summarizer to hold our pipeline and in order to set it up we've written pipeline pipeline and then to that with passive parameter called summarization summarization ."},
 {'summary_text': ' Beautiful

In [77]:
finalSummary = " ".join([summary["summary_text"] for summary in res])

In [78]:
print(finalSummary)

 In this video we're going to be taking a look at how we can use ai to summarize long blog posts . In order to summarize our blog post we're using a library called transformers by a group called hugging face . We  The code for this video is going to be available via GitHub so if you want to pick this up and just run with it really quickly you can go to GitHub . We're going to use beautiful soup to pre-process a blog post  We're going to go through this step by step and actually take a look at how it's done . We'll install our core dependency which is going to be transformers so in order to do that what we've written is exclamation  we actually wrote so what we've done is we've created a new variable called summarizer to hold our pipeline and in order to set it up we've written pipeline pipeline and then to that with passive parameter called summarization summarization .  Beautiful soup can actually use beautiful soup to actually go through all of this text and pull out the specific tag